In [1]:
import datetime as dt
import glob
import os
import shutil

import altair as alt
import aquamonitor as am
import labware as lw
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# alt.data_transformers.disable_max_rows()
alt.data_transformers.enable("json")
plt.style.use("ggplot")
lw.host = am.host = "https://aquamonitor.niva.no/"

In [2]:
# Login to am
am_token = am.login()

Please enter your credentials.


Username:  ···
Password:  ··············


In [3]:
# Login to lw
lw.aqua_site = am.aqua_site = "admin/"
lw_token = am.login()

# Reset am to 'AquaServices'
am.aqua_site = "AquaServices"

Please enter your credentials.


Username:  ···
Password:  ··············


# Elveovervåkingsprogrammet quality control
# Notebook 02: Quality control visualisation

## 1. Define projects and periods of interest

Relevant project codes:

 * Elveovervåkingsprogrammet 2017 - 2020 is project ID 10306 (16384-3)
 * Elveovervåkingsprogrammet 2021 - 2025 is project ID 12564 (200310-03) 

In [4]:
# Define historic project and period of interest for the "reference" dataset
his_proj_ids = [11126, 11927, 12402, 12481]
his_st_dt = "01.01.2017"
his_end_dt = "31.12.2020"

# Define future project and year of interest. NOTE: The Labware/GraphQL
# interface requires the project code (not number), and dashes are usually
# replaced by semi-colons
fut_proj_code = "200311;KJA"
fut_st_dt = "01.01.2021"
fut_end_dt = "31.12.2025"

## 2. Get station details from Aquamonitor

In [5]:
# Get stations for projects
df_list = []
for proj_id in his_proj_ids:
    his_stn_df = am.get_project_stations(proj_id, token=am_token)
    his_stn_df.rename(
        {
            "ProjectId": "project_id",
            "StationId": "station_id",
            "StationCode": "station_code",
            "StationName": "station_name",
            "Type": "type",
        },
        inplace=True,
        axis="columns",
    )
    df_list.append(his_stn_df)
    
his_stn_df = pd.concat(df_list, axis='rows', sort=False).drop_duplicates()
print(f"{len(his_stn_df)} stations in project.")
his_stn_df.head()

187 stations in project.


,project_id,station_id,station_code,station_name,type
0,11126,10183,AKEEGLO3,Vorma v/Svanfossen,Elv
1,11126,40362,NTRENAM,Namsen,Elv
2,11126,40487,SFJESTR,Stryneelva,Elv
3,11126,69417,HEDETRY3,Trysilelva v/Søndre Løberg,Elv
4,11126,69418,HEDETRY1,Trysilelva v/kanalen,Elv


## 3. Get Aquamonitor water chemistry

**Note:** Aquamonitor includes sensor datasets from Alna, Glomma, Drammenselva, Storelva and Målselva that are not present in Labware. As far as I can tell, most of these sensors have separate site codes and the data are therefore *not* included in the query below. However, sensor data from Alna *is* linked to the main RID monitoring station, so data queried from AM includes hourly data from this site. Furthermore, data from the Alna sensor during 2017 seem to be of very poor quality - see e-mail discussions with Roar and Odd Arne on 24.02.2021 for details. Øyvind has suggested removing all sensor data from the comparison anyway (see e-mail received 24.02.2021 16.00).

In [6]:
# Get historic water chemsitry
df_list = []
for proj_id in his_proj_ids:
    am_df = am.get_project_chemistry(proj_id, his_st_dt, his_end_dt, token=am_token)
    df_list.append(am_df)
    
am_df = pd.concat(df_list, axis='rows', sort=False).drop_duplicates()
am_df.to_csv(r"../data/rid_reference_data_option3.csv", index=False)

In [7]:
# Read previously saved data for speed
am_df = pd.read_csv(r"../data/rid_reference_data_option3.csv")
am_df["SampleDate"] = pd.to_datetime(am_df["SampleDate"]).dt.tz_localize(None)
am_df.columns = [
    "project_id",
    "project_name",
    "station_id",
    "station_code",
    "station_name",
    "sample_date",
    "depth1",
    "depth2",
    "parameter",
    "flag",
    "value",
    "unit",
]

# Alna sensor data is very strange and these are the only samples with depth1 = depth2 = 0.5
# Remove these records
am_df = am_df.query("(depth1 == 0) and (depth2 == 0)")

am_df.head()

,project_id,project_name,station_id,station_code,station_name,sample_date,depth1,depth2,parameter,flag,value,unit
0,11126,Elveoverv opsj3 2017,10183,AKEEGLO3,Vorma v/Svanfossen,2017-05-04 14:50:00,0.0,0.0,Ca,NaN,5.84,mg/L
1,11126,Elveoverv opsj3 2017,10183,AKEEGLO3,Vorma v/Svanfossen,2017-05-04 14:50:00,0.0,0.0,Konduktivitet,NaN,4.58,mS/m
2,11126,Elveoverv opsj3 2017,10183,AKEEGLO3,Vorma v/Svanfossen,2017-05-04 14:50:00,0.0,0.0,NH4-N,NaN,7.00,µg/l
3,11126,Elveoverv opsj3 2017,10183,AKEEGLO3,Vorma v/Svanfossen,2017-05-04 14:50:00,0.0,0.0,NO3-N,NaN,380.00,µg/l
4,11126,Elveoverv opsj3 2017,10183,AKEEGLO3,Vorma v/Svanfossen,2017-05-04 14:50:00,0.0,0.0,Nitrogen part,NaN,7.80,µg/l N


## 4. Get Labware water chemistry

In [8]:
# Get Labware projects
proj_df = lw.get_labware_projects(lw_token, fut_proj_code)
proj_df.head()

,name,status,closed
0,1089-9989,U,False
1,1089-9990,U,False
2,1089-9991,U,False
3,1089-9992,U,False
4,1089-10016,P,False


In [9]:
# Get Labware samples
samp_df = lw.get_labware_project_samples(lw_token, proj_df["name"])
samp_df.head()

,sampleNumber,textID,projectStationId,status,sampledDate,sampleDepthUpper,sampleDepthLower,station_id,station_name,station_type
0,133757,NR-2021-00617,57961,A,2021-02-07,0.0,0.0,71949,Trysilelva v/Buflogrenda,Elv
1,133758,NR-2021-00618,57960,A,2021-02-07,0.0,0.0,69417,Trysilelva v/Søndre Løberg,Elv
2,133759,NR-2021-00619,57959,A,2021-02-07,0.0,0.0,69443,Trysilelva v/Skjærholmen,Elv
3,133760,NR-2021-00620,58007,A,2021-02-09,0.0,0.0,73271,Driva v/Steggelbakken,Elv
4,133761,NR-2021-00621,58014,A,2021-02-09,0.0,0.0,70610,Driva v/Gjøra,Elv


In [10]:
# Get results for Labware samples
res_df = lw.get_labware_sample_results(lw_token, samp_df["sampleNumber"])
res_df.head()

,accreditedId,analysis,entryQualifier,loq,mu,name,numericEntry,sample_id,status,test.anaFraction,units
0,NS-EN ISO/IEC 17025:2005 NA TEST 009,KONDUKTIVITET,None,"<0,1",0.0,Konduktivitet,2.26,133757,A,None,MS_P_M
1,None,KONDUKTIVITET,None,None,0.0,Kond_Temp,23.10,133757,A,None,DEGREE_C
2,NS-EN ISO/IEC 17025:2005 NA TEST 009,ANION_KATION,None,"<0,003",0.0,Kalium,0.36,133757,A,None,MG_P_L
3,NS-EN ISO/IEC 17025:2005 NA TEST 009,ANION_KATION,None,"<0,002",0.0,Magnesium,0.85,133757,A,None,MG_P_L
4,NS-EN ISO/IEC 17025:2005 NA TEST 009,ANION_KATION,<,<2,0.0,Ammonium,2.00,133757,A,None,UG_N_L


In [11]:
# Tidy
samp_df2 = samp_df[
    [
        "sampleNumber",
        "textID",
        "station_id",
        "station_name",
        "station_type",
        "sampledDate",
        "sampleDepthUpper",
        "sampleDepthLower",
    ]
]

samp_df2.columns = [
    "sample_id",
    "sample_code",
    "station_id",
    "station_name",
    "station_type",
    "sample_date",
    "depth1",
    "depth2",
]

res_df["test.anaFraction"].replace(
    {
        None: "",
        "Partikulært": "-part",
        "Filtrert": "-filt",
    },
    inplace=True,
)
res_df["name"] = res_df["name"] + res_df["test.anaFraction"]
res_df2 = res_df[
    ["sample_id", "name", "status", "entryQualifier", "numericEntry", "units"]
]
res_df2.columns = ["sample_id", "parameter", "status", "flag", "value", "units"]

# Join
lw_df = pd.merge(res_df2, samp_df2, how="left", on="sample_id")

# Add verbose status codes
res_status = pd.read_csv("../data/labware_result_status_codes.csv", sep=";")
lw_df = pd.merge(lw_df, res_status, how="left", on="status")
del lw_df["status"]
lw_df.rename({"description": "status"}, axis="columns", inplace=True)

# # Get only surface samples
# lw_df = lw_df.query("(depth1==0) and (depth2==0)")
# del lw_df["depth1"], lw_df["depth2"]

# Drop duplicates
lw_df.drop_duplicates(inplace=True)

# Remove strange results where station ID is NaN and sample date is '0001-01-01'
lw_df = lw_df.query("station_id == station_id")

# Parse dates
lw_df["sample_date"] = pd.to_datetime(lw_df["sample_date"])

# Get just data for the period of interest
lw_df = lw_df.loc[
    (lw_df["sample_date"] >= fut_st_dt) & (lw_df["sample_date"] <= fut_end_dt)
]

# Lookup matching Labware and AM pars
par_map = pd.read_csv(r"../data/lw_am_par_map.csv", sep=";", decimal=",")

# Join AM par names
lw_df = pd.merge(
    lw_df,
    par_map,
    left_on=["parameter", "units"],
    right_on=["lw_meth", "lw_unit"],
    how="left",
)

# Convert units
lw_df["value"] = lw_df["value"] * lw_df["lw2am_fac"]

# Drop NaNs and duplicates
cols = [
    "sample_id",
    "sample_code",
    "station_id",
    "station_name",
    "station_type",
    "sample_date",
    "depth1",
    "depth2",
    "am_par",
    "status",
    "flag",
    "value",
    "am_unit",
]
lw_df = lw_df.query("am_par == am_par")
lw_df.drop_duplicates(subset=cols, inplace=True)

# Tidy
lw_df = lw_df[
    [
        "station_id",
        "sample_code",
        "sample_date",
        "depth1",
        "depth2",
        "status",
        "am_par",
        "flag",
        "value",
        "am_unit",
    ]
]

lw_df.rename(
    {
        "am_par": "parameter",
        "am_unit": "unit",
    },
    inplace=True,
    axis="columns",
)

# Join station details
lw_df = pd.merge(
    lw_df,
    his_stn_df[["station_id", "station_code", "station_name"]],
    how="left",
    on="station_id",
)

lw_df["period"] = "New"

# Filter based on status
# lw_df = lw_df.query("status == 'Authorised'")
lw_df = lw_df.query("status != 'Not entered'")

lw_df.head()

,station_id,sample_code,sample_date,depth1,depth2,status,parameter,flag,value,unit,station_code,station_name,period
0,71949,NR-2021-00617,2021-02-07,0.0,0.0,Authorised,Konduktivitet,None,2.26,mS/m,HEDETRY4,Trysilelva v/Buflogrenda,New
1,71949,NR-2021-00617,2021-02-07,0.0,0.0,Authorised,K,None,0.36,mg/L,HEDETRY4,Trysilelva v/Buflogrenda,New
2,71949,NR-2021-00617,2021-02-07,0.0,0.0,Authorised,Mg,None,0.85,mg/L,HEDETRY4,Trysilelva v/Buflogrenda,New
3,71949,NR-2021-00617,2021-02-07,0.0,0.0,Authorised,Magnesium,None,0.85,mg/l,HEDETRY4,Trysilelva v/Buflogrenda,New
4,71949,NR-2021-00617,2021-02-07,0.0,0.0,Authorised,NH4-N,<,2.00,µg/l,HEDETRY4,Trysilelva v/Buflogrenda,New


## 5. Combine datasets

In [12]:
# Standardise dates
# am_df["sample_date"] = am_df["sample_date"].dt.date
# lw_df["sample_date"] = lw_df["sample_date"].dt.date

# Match columns
am_df["sample_code"] = np.nan
am_df["status"] = "Approved"
am_df["period"] = "Reference"
del am_df["project_id"], am_df["project_name"]

# Combine
df = pd.concat([am_df, lw_df], axis="rows")

cols = [
    "period",
    "station_id",
    "station_code",
    "station_name",
    "sample_code",
    "status",
    "sample_date",
    "depth1",
    "depth2",
    "parameter",
    "flag",
    "value",
    "unit",
]
df = df[cols]

# Fill NaN in units with 'None'
df["unit"].fillna("None", inplace=True)

## 6. Subset to parameters of interest

In [13]:
# Remove Turbidity in NTU
df = df.query("unit != 'NTU'")

# Get pars of interest
par_list = [
    "Ag",
    "As",
    "Ca",
    "Cd",
    "Cr",
    "Cu",
    "DOC",
    "Hg",
    "Konduktivitet",
    "Mg",
    "NH4-N",
    "NO3-N",
    "Ni",
    #    "Nitrogen part",
    "PO4-P",
    #    "POC",
    "Pb",
    "STS",
    "Si",
    "TOC",
    "TOTN",
    #    "TOTN (est.)",
    #    "TOTN (old EF)",
    "TOTP",
    "TSM",
    "Temperatur",
    "Turbiditet",
    "Vannstand",
    "Zn",
    "pH",
]
df = df.query("parameter in @par_list")

## 7. Distribution plots

In [14]:
# Set axis scale for plots
ax_scale = "Linear"  # Or 'Log'

In [15]:
# Build drop-down list
df["parameter_unit"] = df["parameter"] + "_" + df["unit"]
par_list = ["None"] + sorted(df["parameter_unit"].unique())
input_dropdown = alt.binding_select(options=par_list)
selection = alt.selection_single(
    fields=["parameter_unit"], bind=input_dropdown, name="Select"
)

In [16]:
# Ticks
ticks = (
    alt.Chart(
        df,
        height=150,
        width=450,
        title="Strip plot",
    )
    .add_selection(selection)
    .transform_filter(selection)
    .mark_tick(
        thickness=2,
        size=30,
        opacity=0.3,
    )
    .encode(
        x=alt.X("value:Q", title="Value", scale=alt.Scale(type=ax_scale.lower())),
        y=alt.Y(
            "period:N",
            title="",
            sort=[
                "Reference",
                "New",
            ],
        ),
        color="period:N",
        tooltip=[
            "period:N",
            "station_id:N",
            "station_code:N",
            "station_name:N",
            "sample_code:N",
            "status:N",
            "sample_date:T",
            "depth1:Q",
            "depth2:Q",
            "parameter:N",
            "flag:N",
            "value:Q",
            "unit:N",
        ],
    )
    .interactive()
)

ticks.configure_axis(
    labelFontSize=16,
    titleFontSize=20,
).configure_legend(labelFontSize=16)

# Q-Q plot
base = alt.Chart(df, height=300, width=450, title="Q-Q plot")

scatter = (
    base.transform_filter(selection)
    .transform_quantile(
        "value",
        step=0.05,
        as_=["percentile", "value"],
        groupby=["period"],
    )
    .transform_pivot("period", groupby=["percentile"], value="value")
    .mark_point()
    .encode(
        x=alt.X(
            "Reference:Q",
            title="Reference data",
            scale=alt.Scale(type=ax_scale.lower()),
        ),
        y=alt.Y("New:Q", title="New data", scale=alt.Scale(type=ax_scale.lower())),
        color=alt.Color("percentile:Q", scale=alt.Scale(scheme="turbo")),
        tooltip=["percentile:Q", "Reference:Q", "New:Q"],
    )
    .interactive()
)

# 1:1 line
line = (
    base.transform_filter(selection)
    .transform_quantile(
        "value",
        step=0.05,
        as_=["percentile", "value"],
        groupby=["period"],
    )
    .transform_pivot("period", groupby=["percentile"], value="value")
    .mark_line()
    .encode(
        x=alt.X("Reference:Q", title="", scale=alt.Scale(type=ax_scale.lower())),
        y=alt.Y("Reference:Q", title="", scale=alt.Scale(type=ax_scale.lower())),
    )
)

qq_plot = scatter + line
qq_plot.configure_axis(
    labelFontSize=16,
    titleFontSize=20,
).configure_legend(labelFontSize=16)

# KDE plot
kde = (
    alt.Chart(
        df,
        height=160,
        width=450,
        title="Density plot",
    )
    .transform_filter(selection)
    .transform_density(
        density="value",
        groupby=["period"],
    )
    .mark_area(
        opacity=0.3,
    )
    .encode(
        x=alt.X("value:Q", title="Value", scale=alt.Scale(type=ax_scale.lower())),
        y=alt.Y("density:Q", title=""),
        color="period:N",
        row=alt.Row(
            "period:N",
            title="",
            sort=[
                "Reference",
                "New",
            ],
        ),
    )
    .interactive()
)

kde.configure_axis(
    labelFontSize=16,
    titleFontSize=20,
).configure_legend(labelFontSize=16)

chart = (ticks & qq_plot) | kde
chart.save("distribution_plots_option3.json")

Issues:

 * Aquamonitor/Labware has both `DOC_mg/L C` and `DOC_mg/l` in this project
 * Nitrogen part_ug/l different in LW vs AM
 * POC_ug/l different in LW vs AM
 * Temperatur_C different in LW vs AM

## 8. Outliers per site

In [17]:
iqr_fac = 1.5

df_list = []
grps = df.groupby(
    ["station_id", "station_code", "station_name", "parameter", "depth1", "depth2"]
)
for idx, grp_df in grps:
    grp_df.set_index("sample_date", inplace=True)
    grp_df.sort_index(inplace=True)
    his_df = grp_df.query("period == 'Reference'").copy()
    new_df = grp_df.query("period == 'New'").copy()

    if (len(his_df) > 10) and (len(new_df) > 0):
        new_df["outlier"] = 0
        uq = his_df["value"].quantile(0.75)
        lq = his_df["value"].quantile(0.25)
        iqr = uq - lq
        new_df.loc[new_df["value"] > uq + iqr_fac * iqr, "outlier"] = 1
        new_df.loc[new_df["value"] < lq - iqr_fac * iqr, "outlier"] = 1

        df_list.append(new_df.query("outlier != 0"))

if len(df_list) > 0:
    out_df = pd.concat(df_list)
    n_stns = len(out_df["station_id"].unique())
    print(
        f"There are {n_stns} stations with new data values that are more than {iqr_fac}*IQR above or below the historic IQR.\n"
    )
    out_csv = f"../output/timerseries_outliers_option3.csv"
    out_df = out_df.reset_index()
    out_df.to_csv(out_csv, index=False)
    out_df.head()

There are 28 stations with new data values that are more than 1.5*IQR above or below the historic IQR.



In [24]:
out_df.head(30)

,station_code,sample_date,depth1,depth2,parameter,outlier
0,NORERAN,2021-04-10,0.0,0.0,Konduktivitet,1
1,NORERAN,2021-04-10,0.0,0.0,Turbiditet,1
2,NORERØS,2021-03-09,0.0,0.0,Konduktivitet,1
3,NORERØS,2021-02-16,0.0,0.0,TSM,1
4,NORERØS,2021-03-09,0.0,0.0,Turbiditet,1
5,NORERØS,2021-02-16,0.0,0.0,pH,1
6,NORERØS,2021-04-10,0.0,0.0,pH,1
7,NTREVER,2021-01-26,0.0,0.0,NH4-N,1
8,NTREVER,2021-02-01,0.0,0.0,TOC,1
9,NTREVER,2021-02-01,0.0,0.0,pH,1


In [18]:
if len(df_list) > 0:
    out_df["temp"] = out_df["station_code"] + "_" + out_df["parameter"]
    df["temp"] = df["station_code"] + "_" + df["parameter"]
    filt = list(out_df["temp"].unique())
    df = df.query("temp in @filt")
    del df["temp"], out_df["temp"]
    out_df = out_df[
        ["station_code", "sample_date", "depth1", "depth2", "parameter", "outlier"]
    ]
    df = pd.merge(
        df,
        out_df,
        on=["station_code", "sample_date", "depth1", "depth2", "parameter"],
        how="left",
    )
    df["outlier"].fillna(0, inplace=True)
    df["series"] = (
        df["station_code"]
        + " "
        + df["parameter"]
        + " ("
        + df["depth1"].astype(int).astype(str)
        + " m, "
        + df["depth2"].astype(int).astype(str)
        + " m)"
    )

    df["vline"] = dt.datetime(*[int(i) for i in fut_st_dt.split(".")][::-1])

    df.head()

In [19]:
if len(df_list) > 0:
    # Build drop-down list
    series_list = ["None"] + sorted(df["series"].unique())
    input_dropdown = alt.binding_select(options=series_list)
    selection = alt.selection_single(fields=["series"], bind=input_dropdown, name="Select")

In [20]:
if len(df_list) > 0:
    base = alt.Chart(df, height=400, width=800, title="Time series plots")
    st = "-".join([i for i in his_st_dt.split(".")][::-1])
    end = "-".join([i for i in fut_end_dt.split(".")][::-1])
    time_domain = pd.to_datetime([st, end]).astype(int) / 10 ** 6
    series = (
        base.add_selection(selection)
        .transform_filter(selection)
        .mark_line(point=True)
        .encode(
            #  x="sample_date:T",
            x=alt.X(
                "sample_date:T", title="Value", scale=alt.Scale(domain=list(time_domain))
            ),
            y="value:Q",
            tooltip=[
                "station_code",
                "sample_date",
                "status",
                "period",
                "depth1",
                "depth2",
                "parameter",
                "unit",
                "value",
                "outlier",
            ],
        )
        .interactive()
    )

    points = (
        base.transform_filter(selection)
        .mark_circle()
        .encode(
            x=alt.X("sample_date:T", title="Value"),
            y="value:Q",
            color=alt.Color(
                "outlier:N", scale=alt.Scale(domain=[0, 1], range=["black", "red"])
            ),
        )
    )

    vline = base.mark_rule(color="red").encode(
        x=alt.X("vline:T", title="Value"),
    )

    chart = series + points + vline
    chart.save("timeseries_plots_option3.json")

## 9. Set date for latest update

In [21]:
def update_html_date(html_file):
    """Update the data in <h3> with the current date."""
    # Build new text
    today = dt.datetime.today()
    today = today.strftime("%d.%m.%Y")
    new_text = f"Labware results were last updated {today}"

    # Update HTML. See https://stackoverflow.com/a/42882971/505698
    soup = BeautifulSoup(open(html_file), "html.parser")
    h3 = soup.find("h3")
    h3.string.replace_with(new_text)

    with open(html_file, "w") as file:
        file.write(str(soup))

In [22]:
# Update dates
update_html_date(r"../pages/option3/distribution_plots_option3.html")
update_html_date(r"../pages/option3/timeseries_plots_option3.html")

## 10. Transfer files

In [23]:
# Delete outdated JSON
flist = glob.glob("../pages/option3/*.json")
for fpath in flist:
    os.remove(fpath)

# Move new files to 'pages' folder
flist = glob.glob("*.json")
for fpath in flist:
    shutil.copy(fpath, "../pages/option3/")
    os.remove(fpath)